Useful reading:  
The illustrated transformer: http://jalammar.github.io/illustrated-transformer/  
Attention is all you need: https://arxiv.org/pdf/1706.03762.pdf  
BERT: https://arxiv.org/pdf/1810.04805.pdf  
A Visual Guide to Using BERT for the First Time: http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/  
Tensorflow hub: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2

In [132]:
import pandas as pd
from bert import tokenization
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow import keras

In [133]:
train_df = pd.read_csv("nlp-getting-started/train.csv")

In [134]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable = True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [151]:
input_ids = []
sequence_ids = []
attention_mask_ids = []
max_len = 512
for text in train_df.text:
    input_tokens = tokenizer.tokenize(text)
    input_tokens = ['[CLS]'] + input_tokens + ['[SEP]']
    padding_len = max_len - len(input_tokens)
    token_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    token_ids += [0] * padding_len
    attention_mask = [1] * len(input_tokens) + [0] * padding_len
    sequence = [1] * max_len
    
    attention_mask_ids.append(attention_mask)
    sequence_ids.append(sequence)
    input_ids.append(ids)

    
sequence_ids = np.array(sequence_ids)
input_ids = np.array(input_ids)
attention_mask_ids = np.array(attention_mask_ids)
input_data = input_ids, attention_mask_ids, sequence_ids
labels = np.array(train_df.target)

In [152]:
max_seq_length = 512
input_tokens_ids = Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
attention_mask_ids= Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")  
sequence_ids = Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")
inputs = [input_tokens_ids, attention_mask_ids, sequence_ids]
pooled_output, sequence_output = bert_layer(inputs)
outputs = Dense(1, activation = 'sigmoid')(sequence_output[:, 0, :])

opt = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = opt, loss = 'binary_crossentropy', metric = 'accuracy')

In [ ]:
model.fit(input_data, labels, epochs = 1)

Train on 7613 samples
